### <span style="color: #477EB0; font-weight:700; font-size: 22px">Imports</span>

General

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from io import StringIO
from collections import Counter

NLP

In [3]:
import nltk
from nltk.corpus import stopwords
import nltk
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob

In [4]:
from better_profanity import profanity
profanity.load_censor_words()

In [5]:
date_stamp = datetime.today().strftime('%d-%b-%y')

### <span style="color: #477EB0; font-weight:700; font-size: 22px">Read in Data</span>

In [6]:
file_path = 'data/blizzard_data.html'

In [7]:
with open(file_path, 'r', encoding='utf-8') as f:
    html_raw = bs(f, 'html.parser')

#### <span style="color: #2c71b0; font-weight:600; font-size: 16px">Find table headers</span>

In [8]:
headers = html_raw.find_all('h2')
print(f'Found {len(headers)} tables, headers are listed below')
[print(x.text) for x in headers];

Found 97 tables, headers are listed below
I. What information do we collect?
II. Why do we process your data?
III. Data sharing and disclosure
IV. What are tracking technologies, and how are they used?
V. Auto-decision making/profiling
VI. What level of security is applied to your personal information?
VII. Right of Access
VIII. More Information
Battle.net Account
Home Address
Account Link
Account Link
Account Link
Security
Gameplay History
Beta Opt In Details
Wallet Information
Current Balances
Ledger Entries
Orders
User Settings
US Block List
US Club List
US Friends
EU Block List
EU Club List
EU Friends
KR Block List
KR Club List
KR Friends
PTR Block List
PTR Club List
PTR Friends
Battle.net Whisper Chat
Overwatch Chat
User Data By Region
Penalty History - EU
Penalty History - KR
Penalty History - TW
Penalty History - US
Activity History
Restriction History
Characters
Chat History
Crafting Orders
Player
Player Lootbox
Player Lootbox Unlock
Player Owl Token Spend
Player Progression Ba

### <span style="color: #477EB0; font-weight:700; font-size: 22px">Parse Data from HTML Tables</span>

#### <span style="color: #2c71b0; font-weight:600; font-size: 16px">Activity History (Logins and Logouts)</span>

In [9]:
# get header object in html
header_obj = html_raw.find(string='Activity History')
# find first table after the header 
html_table = header_obj.find_next('table')
# parse the table
activity_history_df = pd.concat(pd.read_html(StringIO(str(html_table))))
activity_history_df.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
activity_history_df.rename(columns={'time_(utc)':'time',
                                    'geo-ip_location':'location'}, inplace=True)
activity_history_df['time'] = pd.to_datetime(activity_history_df['time'], format='%Y-%m-%d %H:%M:%S')

#### <span style="color: #2c71b0; font-weight:600; font-size: 16px">Ranked Match History</span>

In [10]:
# get header object in html
header_obj = html_raw.find(string='Player Ranked Match History')
# find first table after the header 
html_table = header_obj.find_next('table')
# parse the table
match_history_df = pd.concat(pd.read_html(StringIO(str(html_table))))
match_history_df.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
# remove invalid rows
invalid_rows = match_history_df[match_history_df['leaderboard_region']=='INVALID']
match_history_df.drop(index=invalid_rows.index, inplace=True)
match_history_df.reset_index(drop=True, inplace=True)
match_history_df['match_start_date'] = match_history_df['start_time_epoch_seconds'].apply(lambda x: datetime.fromtimestamp(x))

#### <span style="color: #2c71b0; font-weight:600; font-size: 16px">Battle Pass Progression</span>

In [11]:
# get header object in html
header_obj = html_raw.find(string='Player Battle Pass')
# find first table after the header 
html_table = header_obj.find_next('table')
# parse the table
battle_pass_df = pd.concat(pd.read_html(str(html_table)))
battle_pass_df.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
battle_pass_df['last_claim_time'] = battle_pass_df['last_claim_time'].apply(lambda x: datetime.fromtimestamp(x))

C:\Users\hrowe-hibbert\AppData\Local\Temp\ipykernel_16264\371637749.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  battle_pass_df = pd.concat(pd.read_html(str(html_table)))


#### <span style="color: #2c71b0; font-weight:600; font-size: 16px">Coin Ledger</span>

In [12]:
# get header object in html
header_obj = html_raw.find(string='Ledger Entries')
# find first table after the header 
html_table = header_obj.find_next('table')
# parse the table
coin_ledger_df = pd.concat(pd.read_html(StringIO(str(html_table))))
coin_ledger_df.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
coin_ledger_df['timestamp'] = pd.to_datetime(coin_ledger_df['timestamp'], format='%m/%d/%Y %H:%M:%S')
coin_ledger_df['week_start'] = coin_ledger_df['timestamp'].dt.to_period('W').apply(lambda r: r.start_time)
bal = 0
for i, row in coin_ledger_df.sort_values('timestamp').iterrows():
    bal += row['amount']
    coin_ledger_df.at[i, 'balance'] = bal
coin_ledger_df.sort_values('timestamp', inplace=True)

#### <span style="color: #2c71b0; font-weight:600; font-size: 16px">In-Game Chat History</span>

In [13]:
# get header object in html
header_obj = html_raw.find(string='Overwatch Chat')
# find first table after the header 
html_table = header_obj.find_next('table')
# parse the table
chat_history_df = pd.concat(pd.read_html(StringIO(str(html_table))))
chat_history_df.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
chat_history_df['time'] = chat_history_df['time'].apply(lambda x: datetime.fromtimestamp(x))
chat_history_df['message_id'] = chat_history_df.index.tolist() # assign each message a unique id
chat_history_df.sort_values('time', inplace=True)

### <span style="color: #d63031; font-weight:700; font-size: 22px">Chat Data NLP Analysis</span>

In [14]:
print('chat history start:', chat_history_df['time'].min().strftime('%d %b %Y'))
print('chat history end:', chat_history_df['time'].max().strftime('%d %b %Y'))
print('period: ', chat_history_df['time'].max() - chat_history_df['time'].min())

chat history start: 30 Dec 2023
chat history end: 28 Feb 2024
period:  60 days 02:09:58.173000


In [15]:
# create a dictionary of {message id : message text}
message_dict = chat_history_df.set_index('message_id')['message'].to_dict()
# create a list of message text
messages_list = list(message_dict.values())

#### <span style="color: #d63031; font-weight:600; font-size: 16px">Identify Profanity in Messages</span>

In [16]:
custom_badwords = ['ez', 'ur nana']
profanity.add_censor_words(custom_badwords)

In [17]:
# check for profanity
chat_history_df['contains_profanity'] = chat_history_df['message'].apply(profanity.contains_profanity)
# apply profanity filter (used to extract the profane word itself)
chat_history_df['censored_message'] = chat_history_df['message'].apply(profanity.censor)

In [18]:
# remove false positives
false_positives = [ # list of messages which I feel should not be included
    'whats a kind to a god?',
    'ur lucios throwing lmao',
    'ur front line lmao',
    'fr fr on god?',
    'lmao',
    'sniper kill',
    '*zarya'
]
chat_history_df['contains_profanity'] = chat_history_df[['contains_profanity', 'message']].apply(lambda x: False if x.iloc[1] in false_positives else x.iloc[0], axis=1)

In [19]:
# extract the profane word identified by getting the index of the asterisks in the censored message
# e.g. Raw Message: "thats bullshit"; Censored: "thats ********"
# Start index of "*": 6;
# End index of "*": 0 (index from end of the string)
chat_history_df['censored_start'] = chat_history_df[['contains_profanity', 'censored_message']].apply(lambda x: x.iloc[1].index('*') if x.iloc[0] else None, axis=1)
chat_history_df['censored_end'] = chat_history_df[['contains_profanity', 'censored_message']].apply(lambda x: x.iloc[1][::-1].index('*') if x.iloc[0] else None, axis=1)
# convert index from end of string to index from start of string 
chat_history_df['censored_end'] = chat_history_df[['censored_end', 'message']].apply(lambda x: len(x.iloc[1]) if x.iloc[0]==0 else len(x.iloc[1])-x.iloc[0], axis=1)
# extract the profane word
chat_history_df['profanity'] = chat_history_df[['contains_profanity', 'message', 'censored_start', 'censored_end']].apply(lambda x: None if not x.iloc[0] else x.iloc[1][int(x.iloc[2]):int(x.iloc[3])], axis=1)
# fill nulls NOT NEEDED
# chat_history_df['profanity'] = chat_history_df[['contains_profanity', 'profanity']].apply(lambda x: x.iloc[1] if x.iloc[0] else None, axis=1)
# drop calculation columns
chat_history_df.drop(columns=['censored_message', 'censored_start', 'censored_end'], inplace=True)

In [20]:
chat_history_df[chat_history_df['contains_profanity']].shape

(31, 6)

In [27]:
chat_history_df[['time', 'message', 'message_id', 'contains_profanity', 'profanity']].sample(10)

,time,message,message_id,contains_profanity,profanity
403,2024-02-19 00:26:46.703,no winton,403,False,None
356,2024-02-10 21:44:32.816,gg,356,False,None
514,2024-02-27 22:37:46.613,^^,514,False,None
124,2024-01-08 22:52:59.832,tank diff,124,False,None
268,2024-02-02 22:33:11.164,lol ok cool,268,False,None
444,2024-02-21 23:11:25.511,whats a kind to a god?,444,False,None
121,2024-01-08 22:37:15.414,coolio,121,False,None
88,2024-01-03 20:45:36.160,pizaa on me,88,False,None
452,2024-02-24 00:46:40.117,im digital,452,False,None
449,2024-02-24 00:45:45.095,choose your time wisely,449,False,None


#### <span style="color: #d63031; font-weight:600; font-size: 16px">N-gram Analysis</span>

In [28]:
tokenizer = RegexpTokenizer(r'\w+')

In [29]:
# Function to generate n-grams
def generate_ngrams(messages, n):
    ngrams_list = []
    for message in messages:
        tokens = tokenizer.tokenize(message)
        ngrams_list.extend(ngrams(tokens, n))
    return ngrams_list

In [30]:
all_ngrams = []
for i in range(4):
    ngrams_list = generate_ngrams(messages_list, i+1)
    ngram_frequencies = Counter(ngrams_list)
    nested_list = [[n,count] for n, count in zip(list(ngram_frequencies.keys()), list(ngram_frequencies.values()))]
    all_ngrams.extend(nested_list)

In [31]:
ngram_df = pd.DataFrame(all_ngrams, columns=['tokens', 'count'])
ngram_df['number_of_words'] = ngram_df['tokens'].str.len()
ngram_df['phrase'] = ngram_df['tokens'].str.join(' ')
ngram_df.sort_values('count', ascending=False, inplace=True)

In [32]:
# drop ngrams only seen once
drop_seen_once = ngram_df[ngram_df['count']==1].index
ngram_df.drop(index=drop_seen_once, inplace=True)
ngram_df.reset_index(drop=True, inplace=True)
# try to adjust count by n_words as longer ngrams are less likely 
ngram_df['score'] = ngram_df['count'] * (ngram_df['number_of_words'] *1.3)
ngram_df['rank'] = ngram_df['score'].rank(method='dense', ascending=False).astype(int)

In [33]:
# remove stop word uni-grams
stopword_list = stopwords.words('english')
stopword_unigrams = ngram_df[ngram_df['phrase'].isin(stopword_list)].index
ngram_df.drop(index=stopword_unigrams, inplace=True)

In [34]:
ngram_df.sort_values('score', ascending=False, inplace=True)

In [41]:
ngram_df[ngram_df['number_of_words']==2].sample(5)

,tokens,count,number_of_words,phrase,score,rank
163,"(why, would)",2,2,why would,5.2,14
173,"(on, the)",2,2,on the,5.2,14
53,"(we, can)",4,2,we can,10.4,10
65,"(tank, diff)",3,2,tank diff,7.8,12
128,"(is, my)",2,2,is my,5.2,14


#### <span style="color: #d63031; font-weight:600; font-size: 16px">Sentiment Analysis</span>

In [42]:
# Function to perform sentiment analysis
def analyze_sentiments(messages: dict):
    sentiments = []
    for i, message in messages.items():
        blob = TextBlob(message)
        # Polarity ranges from -1 (negative) to 1 (positive)
        polarity = blob.sentiment.polarity
        # Subjectivity ranges from 0 (objective) to 1 (subjective)
        subjectivity = blob.sentiment.subjectivity
        sentiments.append((i, message, polarity, subjectivity))
    return sentiments

In [43]:
# tweak some of the common acronyms used to improve accuracy
cleaned_message_dict = {k:re.sub('lol|lmao|aha', '', x) for k,x in message_dict.items()}
cleaned_message_dict = {k:re.sub('gl hf', 'good luck, have fun', x) for k,x in cleaned_message_dict.items()}
cleaned_message_dict = {k:re.sub('gg', 'good game', x) for k,x in cleaned_message_dict.items()}

message_sentiments = analyze_sentiments(cleaned_message_dict)

In [44]:
sentiment_messages_df = pd.DataFrame(message_sentiments, columns=['message_id', 'message_nlp_cleaned', 'polarity', 'subjectivity'])
sentiment_df = sentiment_messages_df.groupby(['message_nlp_cleaned'], as_index=False).agg({'polarity':'mean', 'subjectivity':'mean', 'message_id':set})
sentiment_df.sort_values('polarity', ascending=False, inplace=True)
sentiment_df.drop(columns=['message_nlp_cleaned'], inplace=True)

In [45]:
# join sentiment scores to message data
chat_history_sentiment = chat_history_df.merge(sentiment_df.explode('message_id'), how='left', on='message_id', validate='m:1')

#### <span style="color: #d63031; font-weight:600; font-size: 16px">Fill Missing Dates (frequency: day)</span>

In [46]:
# add rows for every day from the min date to max date - this will prevent issues later in tableau
# Tableau is picky about interpolating dates (e.g. for a daily chart you need a data point each day even if 0)
daily_date_range = [d.date() for d in pd.date_range(start=chat_history_df['time'].min(), end=chat_history_df['time'].max(), freq='d')]
date_to_add = []
for d in daily_date_range:
    in_dataset_check = chat_history_df[chat_history_df['time'].dt.date == d]
    if in_dataset_check.empty:
        date_to_add.append(pd.to_datetime(d))
# create data frame of "empty rows"
daily_fill_data = pd.DataFrame(data=date_to_add, columns=['time'])
daily_fill_data['message_id'] = None
daily_fill_data['contains_profanity'] = False
daily_fill_data['profanity'] = None
daily_fill_data['polarity'] = 0.0
daily_fill_data['subjectivity'] = 0.0
# join chat data and empty rows
filled_chat_history = pd.concat([chat_history_sentiment, daily_fill_data])
filled_chat_history['week_start'] = filled_chat_history['time'].dt.to_period('W').apply(lambda r: r.start_time)

### <span style="color: #349857; font-weight:700; font-size: 22px">Activity Data Prep</span>

#### <span style="color: #349857; font-weight:600; font-size: 16px">Create login event data (wide dataset)</span>
This code converts the data from long form (one event per row) to a wide form (LOGIN and LOGOUTS time in each row).

This requires linking up paired events e.g. LOGIN  at 10AM should be paired with the first subsequent LOGOUT).

In some cases there are missing events (e.g. no LOGOUT from the same IP as the LOGIN). When no LOGOUT is found,
the LOGOUT time is added as `missing_data_session_length` hours after LOGIN.

In [47]:
logins = activity_history_df[activity_history_df['activity_type']=='Logged into Overwatch'].reset_index(drop=True)
logouts = activity_history_df[activity_history_df['activity_type']=='Logged out of Overwatch'].copy()

missing_data_session_length = 2.3 # assumed length of sessions with no log out event
log_events_data = []
for i, row in logins.iterrows():
    login_time = row['time']
    logouts_after = logouts[logouts['time'] >= login_time] # all logout events after the login
    first_logout = logouts_after[logouts_after['time']==logouts_after['time'].min()] # earliest logout after the login
    first_logout_same_ip = first_logout[first_logout['ip_address']==row['ip_address']] # IP address of logout must be equal to login IP
    if first_logout_same_ip.empty:
        print(f'Log with no logout on: {login_time}, using average session length.')
        logout_time = login_time + timedelta(hours=missing_data_session_length) # assume session length
    else:
        logout_time = first_logout_same_ip.iloc[0]['time'] # get logout time
    log_events_data.append([i, login_time, logout_time]) # store session values
# create wide data set
log_events_df = pd.DataFrame(log_events_data, columns=['id', 'login_time', 'logout_time'])
log_events_df['duration_seconds'] = (log_events_df['logout_time'] - log_events_df['login_time']).dt.seconds

Log with no logout on: 2023-12-20 21:37:07, using average session length.
Log with no logout on: 2024-01-09 21:27:24, using average session length.


In [82]:
# manually edit anomalies (these are specific occasions when I left my PC turned on with Overwatch running - racking up a very long session length)
# I adjust these down to `missing_data_session_length` hours
# anomaly one
edit_date_in = datetime.strptime('2023-12-16 01:25:04', '%Y-%m-%d %H:%M:%S')
edit_date_out = datetime.strptime('2023-12-16', '%Y-%m-%d').date()
edit_date = (log_events_df['login_time'] == edit_date_in) & (log_events_df['logout_time'].dt.date == edit_date_out)
log_events_df.at[log_events_df[edit_date].index[0], 'duration_seconds'] = (missing_data_session_length * 60)
# anomaly two 
edit_date_in = datetime.strptime('2023-07-15 19:34:14', '%Y-%m-%d %H:%M:%S')
edit_date_out = datetime.strptime('2023-07-16', '%Y-%m-%d').date()
edit_date = (log_events_df['login_time'] == edit_date_in) & (log_events_df['logout_time'].dt.date == edit_date_out)
log_events_df.at[log_events_df[edit_date].index[0], 'duration_seconds'] = (missing_data_session_length * 60)
# anomaly three
edit_date_in = datetime.strptime('2024-02-26 01:44:29', '%Y-%m-%d %H:%M:%S')
edit_date_out = datetime.strptime('2024-02-26', '%Y-%m-%d').date()
edit_date = (log_events_df['login_time'] == edit_date_in) & (log_events_df['logout_time'].dt.date == edit_date_out)
log_events_df.at[log_events_df[edit_date].index[0], 'duration_seconds'] = (missing_data_session_length * 60)
# anomaly four
edit_date_in = datetime.strptime('2024-02-04 01:17:22', '%Y-%m-%d %H:%M:%S')
edit_date_out = datetime.strptime('2024-02-05', '%Y-%m-%d').date()
edit_date = (log_events_df['login_time'] == edit_date_in) & (log_events_df['logout_time'].dt.date == edit_date_out)
log_events_df.at[log_events_df[edit_date].index[0], 'duration_seconds'] = (missing_data_session_length * 60)

#### <span style="color: #349857; font-weight:600; font-size: 16px">Create weekly duration data (long)</span>
This code aggregates the session length to a weekly frequency

In [84]:
weekly_datetime_range = pd.date_range(start=log_events_df['login_time'].min(), end=datetime.strptime('2024-06-01', '%Y-%m-%d'), freq='w-mon')
weekly_dates = [d.date() for d in weekly_datetime_range]

In [85]:
login_weekly_durations = {}
for i, d in enumerate(weekly_dates):
    login_on_after_d = log_events_df['login_time'].dt.date >= d
    login_before_week_end = log_events_df['login_time'].dt.date < (d + timedelta(days=7))
    login_during_week = log_events_df[(login_on_after_d) & (login_before_week_end)].copy()
    login_weekly_durations[d] = login_during_week['duration_seconds'].sum()

In [86]:
weekly_durations_df = pd.DataFrame(login_weekly_durations.values(), index=login_weekly_durations.keys()).reset_index()
weekly_durations_df.columns = ['Week', 'duration_seconds']

#### <span style="color: #349857; font-weight:600; font-size: 16px">Create Hour of Login Data</span>
This code breaks down the number of logins and logouts by hour of the day

In [87]:
hour_of_activity = log_events_df.copy()
hour_of_activity['login_hour'] = hour_of_activity['login_time'].dt.hour
hour_of_activity['logout_hour'] = hour_of_activity['logout_time'].dt.hour

In [88]:
hour_of_login = hour_of_activity.groupby(['login_hour'], as_index=False)['id'].nunique()
hour_of_login.rename(columns={'id':'logins',
                              'login_hour':'hour'}, inplace=True)

In [89]:
hour_of_logout = hour_of_activity.groupby(['logout_hour'], as_index=False)['id'].nunique()
hour_of_logout.rename(columns={'id':'logouts',
                              'logout_hour':'hour'}, inplace=True)

In [90]:
activity_by_hour_df = hour_of_login.merge(hour_of_logout, on='hour', how='outer')

In [91]:
hours_to_fill = [h for h in range(24) if h not in activity_by_hour_df['hour'].unique()]
for h in hours_to_fill:
    activity_by_hour_df.at[len(activity_by_hour_df), 'hour'] = h
activity_by_hour_df.fillna(0, inplace=True)

### <span style="color: #349857; font-weight:700; font-size: 22px">Output Data</span>

#### <span style="color: #349857; font-weight:500; font-size: 15px">Coin Ledger (unused in dashboard)</span>

In [64]:
coin_ledger_df.to_csv(f'output/coin_ledger.csv', index=False)

#### <span style="color: #349857; font-weight:500; font-size: 15px">Battlepass Progression</span>

In [66]:
battle_pass_df.to_csv(f'output/battle_pass.csv', index=False)

#### <span style="color: #349857; font-weight:500; font-size: 15px">Chat History</span>

In [67]:
filled_chat_history.to_csv(f'output/chat_history.csv', index=False)

#### <span style="color: #349857; font-weight:500; font-size: 15px">N-Grams</span>

In [68]:
ngram_df.to_csv(f'output/chat_n_grams.csv', index=False)

#### <span style="color: #349857; font-weight:500; font-size: 15px">Activity History - Login/Out Data</span>

In [83]:
log_events_df.to_csv(f'output/overwatch_playtime.csv', index=False)

#### <span style="color: #349857; font-weight:500; font-size: 15px">Activity History - Weekly Timeseries</span>

In [92]:
weekly_durations_df.to_csv(f'output/overwatch_weekly_playtime.csv', index=False)

#### <span style="color: #349857; font-weight:500; font-size: 15px">Activity History - # Login/Out by Hour (unused in dashboard)</span>

In [93]:
activity_by_hour_df.to_csv(f'output/overwatch_activity_by_hour.csv', index=False)